### 用逻辑回归实现多分类（minist手写数字）

In [1]:
import numpy as np
from scipy.io import loadmat
from scipy.optimize import minimize

In [2]:
data = loadmat('E://GitHub Code/Machine-Learning/02_逻辑回归/Dataset/ex3data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
X = data['X']
y = data['y']
"""
X：5000 * 400，样本是灰度图像20 * 20，所有有400个像素特征
y：5000张图像对应的类别，从1~10
"""
X.shape, y.shape


((5000, 400), (5000, 1))

In [4]:
type(X), type(y)

(numpy.ndarray, numpy.ndarray)

In [5]:
np.unique(data['y'])    # 看下有几类标签

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

##### Hypothesis:

In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

<font size=4>***正则化代价函数***</font><br>
&emsp;&emsp;$J_{(\theta)} = \frac{1}{2m} \sum_{i=1}^m [(h_{\theta}(x^{(i)}) -y^{(i)})^2 + \lambda \sum_{j=1}^n \theta_j^2]$

<font size=4>***Logistic_Regression代价函数***</font><br>
&emsp;&emsp;$ J_{(\theta)} = \frac{1}{m} \sum_{i=1}^m (-y^{(i)}log(h_{\theta}(x^{(i)})) - (1-y^{(i)})log(1 - h_{\theta}(x^{(i)}))) $

<font size=4 color=indianred>***Logistic_Regression的正则化代价函数***</font><br>
&emsp;&emsp;$ J_{(\theta)} = \frac{1}{m} \sum_{i=1}^m [-y^{(i)}log(h_{\theta}(x^{(i)})) - (1-y^{(i)})log(1 - h_{\theta}(x^{(i)}))] + \frac{1}{2m} \lambda \sum_{j=1}^n \theta_j^2 $

In [7]:
def cost(theta, X, y, learningrate):
    theta = np.mat(theta)
    X = np.mat(X)
    y = np.mat(y)

    left = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    right = np.multiply((1-y), np.log(1 - sigmoid(X * theta.T)))
    # 未对theta_0进行正则化
    reg = (learningrate /(2*len(X))) * np.sum(np.power(theta[:, 1:theta.shape[1]], 2))

    return (1 / len(X)) * np.sum(left - right) + reg

由于未对$\theta_0$进行正则，所以<font size=4 color=indianred>***梯度下降***</font> 需要分两种情况：<br>
&emsp;&emsp;$ （1）\theta_0:=\theta_0 - \alpha \frac{\partial J_{(\theta_0)}}{\partial \theta_0} $<br>
&emsp;&emsp;$ （2）\theta_j:=\theta_j - \alpha \frac{\partial J_{(\theta_j)}}{\partial \theta_j} \space\space j\in(1, 401)$

&emsp;&emsp;$ （1-1）\frac{\partial J_{(\theta)}}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^{m} [h_{\theta}(x^{(i)}) - y^{(i)}]x_j^{(i)}$<br>
&emsp;&emsp;$ （1-2）\frac{\partial J_{(\theta)}}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^{m} [h_{\theta}(x^{(i)}) - y^{(i)}]x_j^{(i)} + \frac{\lambda}{m} \theta_j$

In [8]:
def grandient_with_loop(theta, X, y, learningrate):
    """
    用 for 循环实现（1-1）和（1-2）
    :param theta: matrix（1，401）
    :param X: matrix（5000,401）
    :param y: matrix（5000，1）
    :param learningrate:
    :return:（1-1）和（1-2）的计算结果
    """
    theta = np.mat(theta)
    X = np.mat(X)
    y = np.mat(y)

    # 将多维数组拉为一维数组,但还是原来的type
    parameters = int(theta.ravel().shape[1])   # （1,401） -> 401
    grad = np.zeros(parameters)     # 1darray  (401,)

    error = sigmoid(X * theta.T) - y    # (5000, 1)

    for j in range(parameters):
        term = np.multiply(error, X[:, j])      # (5000, 1)

        if (j == 0):
            grad[j] = np.sum(term) / len(X)     # 1个数字
        else:
            grad[j] = np.sum(term) / len(X) + learningrate / len(X) * theta[:, j]   # 1个数字

    return grad   # 1darray (401,)

In [9]:
def gradient(theta, X, y, learningrate):
    """
    用 向量/矩阵方法实现（1-1）和（1-2）
    :param theta: matrix（1，401）
    :param X: matrix（5000,401）
    :param y: matrix（5000，1）
    :param learningrate:
    :return:
    """
    theta = np.mat(theta)
    X = np.mat(X)
    y = np.mat(y)

    error = sigmoid(X * theta.T) - y    # (5000, 1)

    # ((401, 5000) * (5000, 1) / 5000).T = (1, 401)
    # 一个数 * (1, 401) = (1, 401)
    grad = ((X.T * error) / len(X)).T + learningrate / len(X) * theta      # matrix (1, 401)

    return np.array(grad).ravel()   # 1darray (401,)

In [10]:
def one_vs_all(X, y, num_labels, learning_rate):
    """
    由二分类演变的多分类，此例中有10类，需要构建10个分类器。每个分类器在类别 “i” 和不是 “i” 之间决定
    :param X: 2darray (5000, 400)
    :param y: 1darray (5000, 1)
    :param num_labels: 10
    :param learning_rate:
    :return: all_theta: 2darray (10, 401)
    """
    rows = X.shape[0]        # 行 5000
    params = X.shape[1]     # 列 401

    # 10个分类器的theta参数
    all_theta = np.zeros((num_labels, params))      # (10, 401)

    for i in range(1, num_labels+1):
        # inital_theta = np.zeros((params, 1))
        initial_theta = np.zeros(params)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))

        """
        func: 要最小化的目标函数
        x0: 变量的初始猜测值，ndarray, shape=(n, ), n是自变量的数目
        arg: 常数值，传递给目标函数及其导数的额外参数
        method: 求极值的方法
        jac：计算梯度向量的方法
        """
        fmin = minimize(fun=cost, x0=initial_theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1, :] = fmin.x

    return all_theta

In [11]:
X = np.insert(X, 0, values=np.ones(5000), axis=1)   # axis=0代表行，axis=1代表列

theta = np.zeros(X.shape[1] + 1)
num_labels = 10
learning_rate = 1

all_theta = one_vs_all(X, y, num_labels, learning_rate)
all_theta

array([[-1.52375398e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30870219e-03, -1.16722028e-09,  0.00000000e+00],
       [-2.60470074e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.17432181e-03, -4.76439019e-04,  0.00000000e+00],
       [-3.93683638e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.59460833e-05, -2.89233317e-07,  0.00000000e+00],
       ...,
       [-6.68582089e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.45250380e-04,  1.34846923e-05,  0.00000000e+00],
       [-3.90839019e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.61803627e-03,  1.20666439e-04,  0.00000000e+00],
       [-3.45338545e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.41509256e-04,  8.60837605e-06,  0.00000000e+00]])

In [12]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]

    X = np.mat(X)
    all_theta = np.mat(all_theta)

    # （5000, 401) * (401, 10)
    h = sigmoid(X * all_theta.T)    # (5000, 10)

    # np.argmax(array, axis=1): 按行获得数组该行数值最大的那个数的索引
    h_argmax = np.argmax(h, axis=1)

    # 程序中是从第0行开始的，而label是从1开始的
    h_argmax = h_argmax + 1

    return h_argmax

In [13]:
y_pred = predict_all(X, all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(correct) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 94.39999999999999%
